This notebook demonstrates how to train a screen embedding model that outputs an embedding for each screen which can be used to predict if they are `same` or `different` or to compute distance for screen similarity search. The notebook relies on having the WebUI data downloaded, so for this part, run `downloads/downloader.py` again and uncomment 'Optional WebUI dataset' lines. Make sure to comment out the other lines for the files you have already downloaded. 

In [7]:
import os
from screensim.ui_datasets import *
from screensim.ui_models import *
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import *
import torch
import datetime
from pytorch_lightning.loggers import TensorBoardLogger

In [10]:
ARTIFACT_DIR = "./checkpoints_screensim_web7k"
CHECK_INTERVAL_STEPS = 2000

if not os.path.exists(ARTIFACT_DIR):
    os.makedirs(ARTIFACT_DIR)

In [3]:
# Start tensorboard
logger = TensorBoardLogger(ARTIFACT_DIR)

Open `tensorboard --logdir=checkpoints_sim_web7k/lightning_logs/`

In [8]:
data = WebUISimilarityDataModule(split_file="downloads/train_split_web7k.json")

FileNotFoundError: [Errno 2] No such file or directory: 'downloads/train_split_web7k.json'

In [5]:
model = UIScreenEmbedder()

In [6]:
print("***********************************")
print("checkpoints: " + str(os.listdir(ARTIFACT_DIR)))
print("***********************************")

checkpoint_callback = ModelCheckpoint(
    dirpath=ARTIFACT_DIR, every_n_train_steps=CHECK_INTERVAL_STEPS, save_last=True
)
checkpoint_callback2 = ModelCheckpoint(
    dirpath=ARTIFACT_DIR,
    filename="screensim",
    save_top_k=1,
    every_n_train_steps=CHECK_INTERVAL_STEPS,
    mode="max",
    monitor="f1",
)
earlystopping_callback = EarlyStopping(monitor="f1", mode="max", patience=20)

***********************************
checkpoints: ['untitled folder', 'last.ckpt', '.DS_Store', 'lightning_logs']
***********************************


In [7]:
trainer = Trainer(
    accelerator='cpu',
    gradient_clip_val=1.0,
    callbacks=[checkpoint_callback, checkpoint_callback2, earlystopping_callback],
    logger=logger,
    val_check_interval=CHECK_INTERVAL_STEPS,
    accumulate_grad_batches=4,
    limit_val_batches=50,
    max_epochs=100
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/aswearngin/miniforge3/envs/cix-new/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [8]:
trainer.fit(model, data)

/Users/aswearngin/miniforge3/envs/cix-new/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory ./checkpoints_screenrecognition_web7k exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name       | Type       | Params
------------------------------------------
0 | model      | ResNet     | 11.3 M
1 | classifier | Sequential | 131 K 
------------------------------------------
11.4 M    Trainable params
0         Non-trainable params
11.4 M    Total params
45.719    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/aswearngin/miniforge3/envs/cix-new/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/nr/st541_hs0v94j38_dq72_dmm0000gn/T/ipykernel_89250/4116084536.py", line 1, in <module>
  File "/Users/aswearngin/miniforge3/envs/cix-new/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py", line 608, in fit
    call._call_and_handle_interrupt(
  File "/Users/aswearngin/miniforge3/envs/cix-new/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py", line 62, in _call_and_handle_interrupt
    logger.finalize("failed")
  File "/Users/aswearngin/miniforge3/envs/cix-new/lib/python3.10/site-packages/lightning_utilities/core/rank_zero.py", line 42, in wrapped_fn
    return fn(*args, **kwargs)
  File "/Users/aswearngin/miniforge3/envs/cix-new/lib/python3.10/site-packages/pytorch_lightning/loggers/t

FileNotFoundError: [Errno 2] No such file or directory: b'./checkpoints_screenrecognition_web7k/lightning_logs/version_4/events.out.tfevents.1717337654.Amandas-MacBook-Pro-2.local.89250.0'